# SETTING it up 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from sklearn.metrics import  r2_score
from sklearn.model_selection import  train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
import copy
import category_encoders as ce
from sklearn.model_selection import GridSearchCV
from scipy.stats import skew
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

from scipy.stats import skew
from scipy.stats.stats import pearsonr

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, Lasso, ElasticNetCV
from sklearn.model_selection import cross_val_score


import warnings

warnings.filterwarnings('ignore')
from sklearn.preprocessing import PolynomialFeatures



from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error


/Users/richardmason/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv('/Users/richardmason/Downloads/train (1).csv')

test = pd.read_csv('/Users/richardmason/Downloads/test (1).csv')

In [3]:
train_ID = train['Id']
test_ID = test['Id']

In [4]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
All = pd.concat((train, test)).reset_index(drop=True)
All.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(All.shape))

All.columns = [x.replace(' ', '') for x in All.columns]

all_data size is : (2930, 80)


# First Impressions 

In [5]:
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
Id                 2051 non-null int64
PID                2051 non-null int64
MS SubClass        2051 non-null int64
MS Zoning          2051 non-null object
Lot Frontage       1721 non-null float64
Lot Area           2051 non-null int64
Street             2051 non-null object
Alley              140 non-null object
Lot Shape          2051 non-null object
Land Contour       2051 non-null object
Utilities          2051 non-null object
Lot Config         2051 non-null object
Land Slope         2051 non-null object
Neighborhood       2051 non-null object
Condition 1        2051 non-null object
Condition 2        2051 non-null object
Bldg Type          2051 non-null object
House Style        2051 non-null object
Overall Qual       2051 non-null int64
Overall Cond       2051 non-null int64
Year Built         2051 non-null int64
Year Remod/Add     2051 non-null int64
Roof Style         20

In [7]:
train.isnull().sum()

Id                   0
PID                  0
MS SubClass          0
MS Zoning            0
Lot Frontage       330
Lot Area             0
Street               0
Alley             1911
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        22
Mas Vnr Area        22
Exter Qual           0
Exter Cond           0
                  ... 
Half Bath            0
Bedroom AbvGr        0
Kitchen AbvGr        0
Kitchen Qual         0
TotRms AbvGrd        0
Functional           0
Fireplaces           0
Fireplace Qu      1000
Garage Type        113
Garage Yr Blt      114
Garage Finish      114
Garage Cars          1
Garage Area

In [8]:
train.describe()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2051.000000,2.051000e+03,2051.000000,1721.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2029.000000,...,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000,2051.000000
mean,1474.033642,7.135900e+08,57.008776,69.055200,10065.208191,6.112140,5.562165,1971.708922,1984.190151,99.695909,...,93.833740,47.556802,22.571916,2.591419,16.511458,2.397855,51.574354,6.219893,2007.775719,181469.701609
std,843.980841,1.886918e+08,42.824223,23.260653,6742.488909,1.426271,1.104497,30.177889,21.036250,174.963129,...,128.549416,66.747241,59.845110,25.229615,57.374204,37.782570,573.393985,2.744736,1.312014,79258.659352
min,1.000000,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,753.500000,5.284581e+08,20.000000,58.000000,7500.000000,5.000000,5.000000,1953.500000,1964.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129825.000000
50%,1486.000000,5.354532e+08,50.000000,68.000000,9430.000000,6.000000,5.000000,1974.000000,1993.000000,0.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,162500.000000
75%,2198.000000,9.071801e+08,70.000000,80.000000,11513.500000,7.000000,6.000000,2001.000000,2004.000000,161.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2930.000000,9.241520e+08,190.000000,313.000000,159000.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,...,1424.000000,547.000000,432.000000,508.000000,490.000000,800.000000,17000.000000,12.000000,2010.000000,611657.000000


## Doing some feature engineering





In [9]:
All["PoolQC"] = All["PoolQC"].fillna("None")

All["MiscFeature"] = All["MiscFeature"].fillna("None")

All["Alley"] = All["Alley"].fillna("None")

All["Fence"] = All["Fence"].fillna("None")

All["FireplaceQu"] = All["FireplaceQu"].fillna("None")

All["LotFrontage"] = All.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    All[col] = All[col].fillna('None')

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    All[col] = All[col].fillna(0)

for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    All[col] = All[col].fillna(0)

for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    All[col] = All[col].fillna('None')

All["MasVnrType"] = All["MasVnrType"].fillna("None")
All["MasVnrArea"] = All["MasVnrArea"].fillna(0)

All['MSZoning'] = All['MSZoning'].fillna(All['MSZoning'].mode()[0])

All = All.drop(['Utilities'], axis=1)

All["Functional"] = All["Functional"].fillna("Typ")

All['Electrical'] = All['Electrical'].fillna(All['Electrical'].mode()[0])

All['KitchenQual'] = All['KitchenQual'].fillna(All['KitchenQual'].mode()[0])

All['Exterior1st'] = All['Exterior1st'].fillna(All['Exterior1st'].mode()[0])
All['Exterior2nd'] = All['Exterior2nd'].fillna(All['Exterior2nd'].mode()[0])

All['SaleType'] = All['SaleType'].fillna(All['SaleType'].mode()[0])

All['MSSubClass'] = All['MSSubClass'].fillna("None")

#Check remaining missing values if any 
All_na = (All.isnull().sum() / len(All)) * 100
All_na = All_na.drop(All_na[All_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :All_na})
missing_data.head()

All['LotFrontage'] = All['LotFrontage'].fillna(All['LotFrontage'].median())

#MSSubClass=The building class
All['MSSubClass'] = All['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
All['OverallCond'] = All['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
All['YrSold'] = All['YrSold'].astype(str)
All['MoSold'] = All['MoSold'].astype(str)

from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(All[c].values)) 
    All[c] = lbl.transform(list(All[c].values))

# shape        
print('Shape All: {}'.format(All.shape))

All['TotalSF'] = All['TotalBsmtSF'] + All['1stFlrSF'] + All['2ndFlrSF']

numeric_feats = All.dtypes[All.dtypes != "object"].index

Shape All: (2930, 79)


# Transform skewed numerical features, to eliminate the effect of outliers

In [10]:
# Check the skew of all numerical features
skewed_feats = All[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #All[feat] += 1
    All[feat] = boxcox1p(All[feat], lam)
    
#All[skewed_features] = np.log1p(All[skewed_features])


Skew in numerical features: 

There are 61 skewed numerical features to Box Cox transform


## Transform categorical variables with pd.get_dummies, and seperate back out train from test

In [11]:
All = pd.get_dummies(All)
print(All.shape)

train = All[:ntrain]
test = All[ntrain:]

train.shape

test.shape

(2930, 227)


(879, 227)

## Drop saleprice from train

In [12]:
y = pd.DataFrame(data = y_train)

In [13]:
train['SalePrice'] = y

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Columns: 228 entries, 1stFlrSF to SalePrice
dtypes: float64(61), int64(1), uint8(166)
memory usage: 1.3 MB


In [15]:
test.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 2051 to 2929
Columns: 227 entries, 1stFlrSF to SaleType_WD 
dtypes: float64(61), uint8(166)
memory usage: 561.5 KB


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Columns: 228 entries, 1stFlrSF to SalePrice
dtypes: float64(61), int64(1), uint8(166)
memory usage: 1.3 MB


In [17]:
train.drop(columns='SalePrice', inplace=True)

# Pipeline

In [27]:
pipe = Pipeline([
    ('pf', PolynomialFeatures(degree= 2, interaction_only=True, include_bias=True)),
    ('rs', RobustScaler(with_centering=True, with_scaling=True)),
    ('lasso', Lasso(normalize = True))])
    


    
    
    
    
   



# Gridsearch

In [28]:
# pipe.fit(train, y_train)

In [29]:
# pipe.score(train,y_train)

In [30]:
# pipe.fit(train, y_train)

In [31]:
gs = GridSearchCV(pipe,param_grid = {},verbose = 1, n_jobs= -1, cv=3)

In [32]:
gs.fit(train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   38.5s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('pf', PolynomialFeatures(degree=2, include_bias=True, interaction_only=True)), ('rs', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

# Scoring

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
gs.best_estimator_

# Streamlined Submitting to Kaggle

In [ ]:
real_preds = gs.predict(test)

In [ ]:
real_preds.shape

In [ ]:
real_test_dic = {'Id':test_ID,
                 'SalePrice': real_preds}

Submission9 = pd.DataFrame(data=real_test_dic)

In [ ]:
import datetime

now = datetime.datetime.now()

now = str(now)

now

Submission_ = Submission9.to_csv(f'/Users/richardmason/Documents/submission{now}.csv', index=False)